In [1]:
%matplotlib qt5
%load_ext autoreload
%autoreload 2
import os, sys, fnmatch
import cv2, time, math, pickle
import numpy as np
import argparse, pprint

from matplotlib import pyplot as plt
import matplotlib
from matplotlib.ticker import NullFormatter
from matplotlib.transforms import Bbox
import matplotlib.gridspec as gridspec

import ipywidgets as widgets
from IPython.display import display, HTML, Math
from ipywidgets import interact, interactive, IntSlider, Layout, interact_manual,interact, HBox, Layout,VBox

# import pandas as pd
# from pandas import DataFrame
# from sklearn import linear_model
# import statsmodels.api as sm

pp = pprint.PrettyPrinter(indent=4)
homeDir = os.path.abspath('')

In [33]:
from scripts.hyutils.sys_utils import *
from scripts.hyutils.img_utils import *
from scripts.hyutils.debug_utils import *
from scripts.hyutils.plotting_utils import *
from scripts.hyutils.uv_mapping_utils import *

In [3]:
def test_intersection(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

def union(a,b):
    x = min(a[0], b[0])
    y = min(a[1], b[1])
    w = max(a[0]+a[2], b[0]+b[2]) - x
    h = max(a[1]+a[3], b[1]+b[3]) - y
    return (x, y, w, h)

In [94]:
from scripts.UVMappingPipeline import UVMappingPipeline
from scripts.VBOATS import VBOATS

pipe = UVMappingPipeline()
vboat = VBOATS()

vboat.testMaxGndStep = 16

In [175]:
# found_imgs, _ = grab_dir_images("/home/hunter/data/images/test_cases")
# found_imgs, _ = grab_dir_images("/home/hunter/devel/vision_playground/test/frames")
# found_imgs, _ = grab_dir_images("/home/hunter/data/images/obstacles")
# found_imgs, _ = grab_dir_images("/home/hunter/data/rosbags/extracted/depth")
found_imgs, _ = grab_dir_images("/home/hunter/data/vboats/test_pool_1")

# found_imgs, _ = grab_dir_images("/home/hunter/data/vboats/tests/ng/huh/depth")
# found_imgs, _ = grab_dir_images("/home/hunter/data/vboats/tests/ng/test_0/depth")
# found_imgs, _ = grab_dir_images("/home/hunter/data/vboats/tests/ng/test_1/depth")
# found_imgs, _ = grab_dir_images("/home/hunter/data/vboats/tests/ng/test_2/depth")
# found_imgs, _ = grab_dir_images("/home/hunter/data/vboats/tests/ng/test_3/depth")
# found_imgs, _ = grab_dir_images("/home/hunter/data/vboats/tests/ng/test_4/depth")
# found_imgs, _ = grab_dir_images("/home/hunter/data/vboats/tests/ng/test_5/depth")
# found_imgs, _ = grab_dir_images("/home/hunter/data/vboats/tests/ng/test_6/depth")

# found_imgs, _ = grab_dir_images("/home/hyoung/Documents/rosbags/extracted/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/devel/vision_playground/test/iai/obstacles")

# found_imgs, _ = grab_dir_images("/home/hyoung/data/d415/tests/tmp/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/d415/tests/distance_test/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/d415/tests/angle_test/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/d415/tests/misc_test/depth")

# found_imgs, _ = grab_dir_images("/home/hyoung/data/vboats")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/vboats/test_cases")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/vboats/test_keyParams")

# found_imgs, _ = grab_dir_images("/home/offset/Pictures/vboats/test_cases")
# found_imgs, _ = grab_dir_images("/home/offset/Pictures/vboats/aion")
# found_imgs, _ = grab_dir_images("/home/offset/Pictures/vboats/aion")
# found_imgs, _ = grab_dir_images("/home/offset/Pictures/vboats/aion/huh")

# found_imgs, _ = grab_dir_images("/home/hyoung/data/vboats/data/tests/ng/huh/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/vboats/data/tests/ng/test_0/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/vboats/data/tests/ng/test_1/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/vboats/data/tests/ng/test_2/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/vboats/data/tests/ng/test_3/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/vboats/data/tests/ng/test_4/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/vboats/data/tests/ng/test_5/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/vboats/data/tests/ng/test_6/depth")

sFImgs = sorted(found_imgs)

In [176]:
style = {'description_width': 'initial'}
layStretch = Layout(flex='1 1 auto')
layCenter = Layout(align_self='center',display = 'flex',align_items='stretch',flex='1 1 auto')
layWidth = Layout(width="auto",align_self='center')

imgPathSelector = widgets.Select(options=sFImgs,value=str(sFImgs[0]),description='File:', layout=Layout(width='100%'))

lblSz1 = widgets.Label(value="Deadzone Sizes (U,V):", layout=layCenter)
sz1Selector = widgets.IntSlider(value=0,min=0,max=20,step=1,description=' ', layout = layCenter)
sz2Selector = widgets.IntSlider(value=3,min=0,max=20,step=1,description=' ', layout = layCenter)

lblSz2 = widgets.Label(value="Strip[0] Deadzone Sizes (U,V):", layout=layCenter)
sz3Selector = widgets.IntSlider(value=4,min=0,max=20,step=1,description=' ', layout = layStretch)
sz4Selector = widgets.IntSlider(value=5,min=0,max=20,step=1,description=' ', layout = layStretch)

lblU = widgets.Label(value="UMap Strip Thresholds (0,1):", layout=layCenter)
u1threshSelector = widgets.FloatSlider(value=0.1,min=0,max=1,step=0.01,description=' ',layout= layStretch)
# u1threshSelector = widgets.FloatSlider(value=0.15,min=0,max=1,step=0.01,description=' ',layout= layStretch)
u2threshSelector = widgets.FloatSlider(value=0.1,min=0,max=1,step=0.01,description=' ',layout = layStretch)

lblV = widgets.Label(value="VMap Strip Thresholds (0,1):", layout=layCenter)
v1threshSelector = widgets.IntSlider(value=5,min=0,max=255,step=1,description=' ',layout = layStretch)
v2threshSelector = widgets.IntSlider(value=70,min=0,max=255,step=1,description=' ',layout = layStretch)

lblD = widgets.Label(value="Deadzone Thresholds (U1, U2, V):", layout=layCenter)
d1threshSelector = widgets.FloatSlider(value=0.25,min=0,max=1,step=0.01,description=' ', layout = layStretch)
d2threshSelector = widgets.FloatSlider(value=0.15,min=0,max=1,step=0.01,description=' ', layout = layStretch)
d3threshSelector = widgets.FloatSlider(value=0.15,min=0,max=1,step=0.01,description=' ', layout = layStretch)

lblK = widgets.Label(value="Kernel Sizes:", layout=layCenter)
k11Selector = widgets.IntSlider(value=1,min=0,max=100,step=1,description='k11', layout = layStretch)
k12Selector = widgets.IntSlider(value=2,min=0,max=100,step=1,description='k12', layout = layStretch)
k21Selector = widgets.IntSlider(value=50,min=0,max=100,step=1,description='k21', layout = layStretch)
k22Selector = widgets.IntSlider(value=5,min=0,max=100,step=1,description='k22', layout = layStretch)
k31Selector = widgets.IntSlider(value=50,min=0,max=100,step=1,description='k31', layout = layStretch)
k32Selector = widgets.IntSlider(value=1,min=0,max=100,step=1,description='k32', layout = layStretch)

win1Selector = widgets.IntSlider(value=10,min=0,max=100,step=1,description='win1', layout = layStretch)
win1ThreshSelector = widgets.FloatSlider(value=0.15,min=0,max=1,step=0.01,description=' ', layout = layStretch)


flag1 = widgets.Checkbox(value=True, description='Use Test Pipeline', layout = layCenter)
flag2 = widgets.Checkbox(value=False, description='Show Computational Timing', layout = layCenter)
flag3 = widgets.Checkbox(value=False, description='Use Morphs', layout = layCenter)

In [177]:
def thresholding(_img, dx=0,dy=3, sz1=5,sz2=10,tu0=0.15,tu1=0.1,tv0=5, tv1=70, tdu = 0.4, tdu2 = 0.1, tdv = 0.15,
    k11=15,k12=2,k21=50,k22=5,k31=75,k32=1, win1=5, win1thr = 0.1, searchWin=10, flag_test=True, flag_timing=False,flag_morph=True
):
#     global overlay
    
    if(flag_test):
        vboat.dW = sz2;        vboat.dH = sz1
        vboat.dead_x = dx;     vboat.dead_y = dy
        vboat.dThs = [tdu,tdu2,win1thr,tdv]
        vboat.dHplus = win1
        
        vboat.kszs[0] = (k11,k12)
        vboat.kszs[1] = (k21,k22)
        vboat.kszs[2] = (k31,k32)
        vboat.testLowCntThresh = k21
        vboat.testHighCntThresh = k31
        vboat.testRestThreshRatio = tdu
        vboat.flag_morphs = flag_morph
        
        vboat.vmap_search_window_height = searchWin
        
        vboat.pipelineTest(_img, threshU1=tu0,threshU2=tu1, threshV1=tv0, threshV2=tv1, timing=flag_timing)
        display_obstacles = cv2.cvtColor(vboat.img, cv2.COLOR_GRAY2BGR)
        display_obstacles = cv2.applyColorMap(display_obstacles,cv2.COLORMAP_JET)
        
        for ob in vboat.obstacles:
            cv2.rectangle(display_obstacles,ob[0],ob[1],(255,255,255),1)
        
        overlay = make_uv_overlay(display_obstacles,vboat.umap_deadzoned,vboat.vmap_raw)
        overlay = cv2.cvtColor(overlay,cv2.COLOR_BGR2RGB)
        pplot(overlay,'Overlay')

        dispU1,dispU2,compU = vboat.umap_displays()
        dispV1,dispV2,compV = vboat.vmap_displays()

        dispU2 = cv2.cvtColor(dispU2,cv2.COLOR_BGR2RGB)
        dispV2 = cv2.cvtColor(dispV2,cv2.COLOR_BGR2RGB)
        pplot(dispU2,'Umap Display')
        pplot(dispV2,'Vmap Display')
        pplot(vboat.vmap_processed,"Processed Vmap")
        
        test_vstrip_filter()
#         _,strip, nStrip,_, disp = vboat.test_filter_first_umap_strip(vboat.stripsU_raw[0],51,[1,1,1,1,1,1], ratio_thresh=vboat.testRestThreshRatio)
#         pplots([strip,nStrip,disp],"Compare",(3,1))
        
        distances = []
        angles = []
        umap = vboat.umap_raw
        xs = vboat.xBounds
        ds = np.array(vboat.dbounds)
        obs = vboat.obstacles
        nObs = len(ds)
        if(nObs is not 0):
            for i in range(nObs):
                disparities = ds[i]
                us = [obs[i][0][0], obs[i][1][0]]
                vs = [obs[i][0][1], obs[i][1][1]]
                z,ux,uy,uz = vboat.calculate_distance(umap,us,disparities,vs)

                theta = math.acos((uz/z))

                distances.append(z)
                angles.append(theta)
                print("Obstacle [%d] Distance, Angle: %.3f, %.3f" % (i,z,np.rad2deg(theta)))
        
    else:
        vboat.pipeline(_img, threshU1=tu0,threshU2=tu1, threshV1=tv0, threshV2=tv1, timing=flag_timing)
        display_obstacles = cv2.cvtColor(vboat.img, cv2.COLOR_GRAY2BGR)

        for ob in vboat.obstacles:
            cv2.rectangle(display_obstacles,ob[0],ob[1],(150,0,0),1)
        pplot(display_obstacles,'Pipeline Obstacles')

In [178]:
%reload_ext autoreload
plt.close('all')

p = interactive(thresholding,
    _img = imgPathSelector, dx = sz1Selector, dy=sz2Selector, sz1=sz3Selector,sz2=sz4Selector,
    tu0 = u1threshSelector,tu1 = u2threshSelector, tv0 = v1threshSelector, tv1 = v2threshSelector,
    tdu = d1threshSelector, tdu2=d2threshSelector, tdv = d3threshSelector, k11=k11Selector,k12=k12Selector,
    k21=k21Selector,k22=k22Selector,k31=k31Selector,k32=k32Selector,win1=win1Selector,win1thr=win1ThreshSelector,
    searchWin=(0,40,1), flag_test = flag1, flag_timing = flag2, flag_morph = flag3
)

rows = []
rows.append(HBox([p.children[0]]))
rows.append(HBox([lblSz1,p.children[1], p.children[2]], layout = layStretch))
rows.append(HBox([lblSz2,p.children[3], p.children[4]], layout = layStretch))
rows.append(HBox([lblU, p.children[5], p.children[6]], layout = layStretch))
rows.append(HBox([lblV, p.children[7], p.children[8]], layout = layStretch))
rows.append(HBox([lblD, p.children[9], p.children[10], p.children[11]], layout = layStretch))
rows.append(HBox([p.children[12], p.children[13]], layout = layStretch))
rows.append(HBox([p.children[14], p.children[15]], layout = layStretch))
rows.append(HBox([p.children[16], p.children[17]], layout = layStretch))
rows.append(HBox([p.children[18], p.children[19]], layout = layStretch))

rows.append(HBox([p.children[-4], p.children[-3], p.children[-2]], layout = layCenter))

controls = VBox(rows, layout = Layout(display='flex'))
output = p.children[-1]

tabs = widgets.Tab([controls, output])
tabs.set_title(0, 'Parameters')
tabs.set_title(1, 'StdOuts')
display(tabs)

VGFiKGNoaWxkcmVuPShWQm94KGNoaWxkcmVuPShIQm94KGNoaWxkcmVuPShTZWxlY3QoZGVzY3JpcHRpb249dSdGaWxlOicsIGxheW91dD1MYXlvdXQod2lkdGg9dScxMDAlJyksIG9wdGlvbnPigKY=


In [12]:
vboat.debug_obstacle_search = False
vboat.debug = False

In [ ]:
print(vboat.dbounds)
pplot(vboat.umap_raw,0)

In [174]:
def test_vstrip_filter():
    vmap = np.copy(vboat.vmap_raw)
    vmax = np.max(vmap)

    testStrips = []
    stripsPv = []
    threshsV = [0, 70, 40,60,60]
    nThreshsV = int(math.ceil((vboat.dmax/256.0) * len(threshsV)))


    stripsV = strip_image(vboat.vmap_deadzoned, nstrips=nThreshsV, horizontal_strips=False)
    pplots(stripsV,"Raw Vmap Strips",(1,nThreshsV))

    # RAW VStrip 1
    testStrips.append(stripsV[0])

    stripMax = np.max(stripsV[0])
    relRatio = stripMax/float(vmax)


    for i, strip in enumerate(stripsV):
        _, tmpStrip = cv2.threshold(strip, threshsV[i], 255, cv2.THRESH_TOZERO)
        stripsPv.append(tmpStrip)

#     pplots(stripsPv,"Pre-Filtered Vmap Strips",(1,nThreshsV))

    vsThs = np.array([
        [0.25, 0.0],
        [0.5, 0.0]
    ])

    # Filtered VStrip 1 STEP 0
    testStrips.append(np.copy(stripsPv[0]))

    tmpMax = np.max(stripsPv[0])

    # Filtered VStrip 1 STEP 1
    testStrips.append(np.copy(stripsPv[0]))

    tmpStrip = stripsPv[0]
    hs, ws = tmpStrip.shape[:2]

    tmpdead = tmpStrip[:, 0:3]
    tmprest = tmpStrip[:, 3:ws]

    deadMax = np.max(tmpdead)
    restMax = np.max(tmprest)
    dmRatio = deadMax / float(stripMax)
    rmRatio = restMax/float(stripMax)
    print("stripMax,vmapMax,deadMax,restMax: %d, %d, %d, %d" % (stripMax,vmax,deadMax,restMax))
    print("relRatio, dmRatio,rmRatio: %.3f, %.3f, %.3f" % (relRatio, dmRatio,rmRatio))
#     pplots([tmpdead,tmprest], "Pre-CLAHE Vmap", (1,2))

    # _, fa = cv2.threshold(tmpdead, tfa,255,cv2.THRESH_TOZERO) #THRESH_TOZERO

    tCLAH = cv2.createCLAHE(clipLimit=2.0,tileGridSize=(5,1))
    tCLAH2 = cv2.createCLAHE(clipLimit=2.0,tileGridSize=(hs,3))
    dCl = tCLAH.apply(tmpdead)
    dCl2 = tCLAH2.apply(tmprest)

#     pplots([dCl,dCl2],"DCLs V",(1,2))
    pc1 = dCl[:, 0:1]
    pc2 = dCl[:, 1:dCl.shape[1]]

    # _,pc1 = cv2.threshold(pc1, 200,255,cv2.THRESH_TOZERO)
    _,pc1 = cv2.threshold(pc1, 150,255,cv2.THRESH_TOZERO)
    _,pc2 = cv2.threshold(pc2, 0,255,cv2.THRESH_TOZERO)
    tmpdead = np.concatenate((pc1,pc2), axis=1)

#     pplots([pc1,pc2],"dCl strips",(1,2))


    cldMax = np.max(tmpdead)
    clrMax = np.max(dCl2)



    # deadThreshGain = (stripMax - deadMax)/255.0
    # deadThreshOffset = int(base_thresh*deadThreshGain)
    # if not vboat.is_ground_present: deadThresh = deadThreshOffset
    # else: deadThresh = base_thresh + deadThreshOffset

    # dead_strip[dead_strip < deadThresh] = 0
    # deadMax = np.max(dead_strip)


    deadMaxGain = (cldMax - deadMax)/float(cldMax)
    restMaxGain = (clrMax - restMax)/float(clrMax)
    print("cldMax,clrMax,deadMaxGain,restMaxGain: %d, %d, %.3f, %.3f" % (cldMax,clrMax,deadMaxGain,restMaxGain))
    newStripMax = int(stripMax*restMaxGain);        newRmRatio = (restMax)/float(newStripMax)
    print("newStripMax, newRmRatio: %d, %.3f" % (newStripMax,newRmRatio))

    ratio_thresh = 0.25;   base = 64
    g1 = restMaxGain*ratio_thresh;         g2 = (g1+ratio_thresh);         g3 = 1 - g1;         g4 = 1 - g2
    t1 = int(g1*clrMax);      t2 = int(g2*clrMax);      t3 = int(g3*clrMax);      t4 = int(g4*clrMax)
    t21 = int(g1*base);       t22 = int(g2*base);       t23 = int(g3*base);       t24 = int(g4*base)
    t31 = int((g2+g2*restMaxGain)*clrMax)
    print; print("g1,g2,g3,g4:\t%.3f, %.3f, %.3f, %.3f" % (g1,g2,g3,g4));
    print("t1,t2,t3,t4:\t%d, %d, %d, %d" % (t1,t2,t3,t4))
    print("t21,t22,t23,t24,t31:\t%d, %d, %d, %d, %d" % (t21,t22,t23,t24,t31));print

    _,tmprest = cv2.threshold(dCl2, t31,255,cv2.THRESH_TOZERO)
    stripsPv[0] = np.concatenate((tmpdead,tmprest), axis=1)


    # Filtered VStrip 1 STEP 2
    testStrips.append(np.copy(stripsPv[0]))

    dw = 5;        vdTh = [0.15, 0]

    # dead_strip = stripsPv[0][:, 0:dw]
    # rest_strip = stripsPv[0][:,dw:stripsPv[0].shape[1]]

    # print("[INFO] Shapes stripsPv[0], dead_strip, rest_strip: %s, %s, %s" % (str(stripsPv[0].shape), str(dead_strip.shape),str(rest_strip.shape)))

    # dead_strip[dead_strip < tmpMax*vdTh[0]] = 0
    # rest_strip[rest_strip < tmpMax*vdTh[1]] = 0
    # stripsPv[0] = np.concatenate((dead_strip,rest_strip), axis=1)

    # # Filtered VStrip 1 STEP 3
    # testStrips.append(np.copy(stripsPv[0]))

    # print("[INFO] stripsPv[0] shape: %s" % (str(stripsPv[0].shape)))

    # stripsPv[1][stripsPv[1] < np.max(stripsPv[1]*0.9)] = 0
    tmpMax = np.max(stripsPv[1])
    tH = stripsPv[1].shape[0]
    top_strip = stripsPv[1][0:tH/3, :]
    bot_strip = stripsPv[1][tH/3:tH, :]

    top_strip[top_strip < tmpMax*vsThs[1,0]] = 0
    bot_strip[bot_strip < tmpMax*vsThs[1,1]] = 0
    stripsPv[1] = np.concatenate((top_strip,bot_strip), axis=0)

    newV = np.concatenate(stripsPv, axis=1)

#     pplots(testStrips,"1st Vmap Strip Filter Steps",(1,len(testStrips)))

    tmp = np.copy(newV)
    vmap = cv2.bitwise_and(tmp,tmp,mask=cv2.bitwise_not(vboat.vmask))
    pplot(vmap,"New Vmap")
    obs, obsU, ybounds, dbounds, windows, nObs = vboat.find_obstacles(vmap, vboat.disparityBounds, vboat.xBounds, ground_detected=vboat.is_ground_present,verbose=True)

    dispV = cv2.applyColorMap(np.copy(vmap),cv2.COLORMAP_JET)
    for wins in windows:
        for win in wins:
            cv2.rectangle(dispV,win[0],win[1],(0,255,255), 1)
    dispV = cv2.cvtColor(dispV,cv2.COLOR_BGR2RGB)
    pplot(dispV,"dispV")


    displayObs = cv2.cvtColor(np.copy(vboat.img), cv2.COLOR_GRAY2BGR)
    displayObs = cv2.applyColorMap(displayObs,cv2.COLORMAP_JET)

    for ob in obs:
        cv2.rectangle(displayObs,ob[0],ob[1],(255,255,255),1)


    overlay = cv2.cvtColor(displayObs,cv2.COLOR_BGR2RGB)
    pplot(overlay,'Test Obs')

In [151]:
print(windows[0])

[[(42, 120), (86, 100)], [(42, 140), (86, 120)], [(42, 160), (86, 140)], [(42, 220), (86, 200)], [(42, 240), (86, 220)]]


In [85]:
verbose = True;     n = 3
umap = np.copy(vboat.umap_raw);   stripsU = strip_image(umap, nstrips=n);    strip = np.copy(stripsU[0])
# Stripping
hs, ws = strip.shape[:2];              dh = hs / n
print("strip.shape: ",strip.shape[:2])
# pplot(strip,"strip")
dead_strip = strip[0:dh, :];           oDead = np.copy(dead_strip)
rest_strip = strip[dh:hs, :];          oRest = np.copy(rest_strip)
print("Shapes -- deadStrip, restStrip: %s, %s" % (str(dead_strip.shape),str(rest_strip.shape)))

hd, wd = dead_strip.shape[:2]
hr, wr = rest_strip.shape[:2]

umapMax = np.max(umap)
stripMax = np.max(strip)
restMax = np.max(rest_strip)

ksz2 = (75,2)
ksz1 = (30,1)
kernelr = cv2.getStructuringElement(cv2.MORPH_RECT,ksz1)
kernelD = cv2.getStructuringElement(cv2.MORPH_RECT,ksz2)

tfa = int(0.5*umapMax);
print("tfa: %d" %(tfa))

tmpdead = dead_strip[0:3, :]
tmprest = dead_strip[3:hd, :]

tCLAH = cv2.createCLAHE(clipLimit=2.0,tileGridSize=(1,20))
dCl = tCLAH.apply(tmpdead)

print("dCl Shape: %s" % str(dCl.shape))


pc1 = dCl[0:1, :]
pc2 = dCl[1:dCl.shape[0], :]
pplots([pc1,pc2],"dCl strips",(2,1))
print("[DEBUG] test_filter_first_umap_strip() -----  Shapes: pc1 [%s], pc2 [%s]" % (str(pc1.shape),str(pc2.shape)))


_,pc1 = cv2.threshold(pc1, 256,255,cv2.THRESH_TOZERO)
_,pc2 = cv2.threshold(pc2, 180,255,cv2.THRESH_TOZERO)

tmpdead2 = np.concatenate((pc1,pc2), axis=0);

# pplots([dCl,dCl3],"dCl3",size=(2,1),flag_resize=True,scale=(5,1))
# pplots([tmpdead,dCl,dCl2],"TEST", (3,1))


_, fa = cv2.threshold(tmpdead, tfa,255,cv2.THRESH_TOZERO) #THRESH_TOZERO

mask = cv2.morphologyEx(fa, cv2.MORPH_CLOSE, kernelD)
tmpdead2 = cv2.morphologyEx(tmpdead2, cv2.MORPH_CLOSE, kernelr)
tmprest = cv2.morphologyEx(tmprest, cv2.MORPH_CLOSE, kernelD)

dead_strip = np.concatenate((tmpdead2,tmprest), axis=0);
# dead_strip = np.concatenate((fa,tmprest), axis=0);
# dead_strip = np.concatenate((mask,tmprest), axis=0);


oStrip = np.concatenate((dead_strip,rest_strip), axis=0);
oDead2 = np.copy(dead_strip)

# pplots([oDead,newdead,trip],"oDead", (3,1))
pplot(oStrip,"oStrip")

# stripMax = np.max(oStrip)
deadMax = np.max(dead_strip)

# Initial Stats
relRatio = stripMax/float(umapMax)
dmRatio = deadMax / float(stripMax)
rmRatio = restMax/float(stripMax)
print("relRatio, dmRatio,rmRatio: %.3f, %.3f, %.3f" % (relRatio, dmRatio,rmRatio))
# Deadzone Filtering
threshD = 50;         base_thresh = 35
deadThreshGain = (stripMax - deadMax)/255.0;        deadThreshOffset = int(base_thresh*deadThreshGain)
if not vboat.is_ground_present: deadThresh = deadThreshOffset
else: deadThresh = base_thresh + deadThreshOffset
dead_strip[dead_strip < deadThresh] = 0
# dead_strip[dead_strip < deadThresh] = 0
if verbose: print("[Deadzone Filtering] base, gain, offset, thresh: %d, %.2f, %d, %d" % (base_thresh,deadThreshGain,deadThreshOffset,deadThresh))
deadMax = np.max(dead_strip)
# Perform CLAHE

# claheDead = cv2.createCLAHE(clipLimit=2.0,tileGridSize=(hd,wd))
claheDead = cv2.createCLAHE(clipLimit=2.0,tileGridSize=(hd,wd))
claheRest = cv2.createCLAHE(clipLimit=2.0,tileGridSize=(hr/2,wr))
clD = claheDead.apply(dead_strip);         copyCld = np.copy(clD)
clR = claheRest.apply(rest_strip);         copyClr = np.copy(clR)
_,stripD = cv2.threshold(clD, threshD,255,cv2.THRESH_TOZERO)                # Filter CLAHE Deadzone
cldMax = np.max(stripD);    clrMax = np.max(clR)                            # CLAHE Maxes
if deadMax == 0: newDeadMax = 1
else: newDeadMax = deadMax
deadMaxGain = (cldMax - deadMax)/float(cldMax)
restMaxGain = (clrMax - restMax)/float(clrMax)
newStripMax = int(stripMax*restMaxGain);        newRmRatio = (restMax)/float(newStripMax)
print("newStripMax, newRmRatio: %d, %.3f" % (newStripMax,newRmRatio))
# Remainder Filtering
ratio_thresh = 0.25;   base = 64
print("stripMax,umapMax,deadMax,restMax: %d, %d, %d, %d" % (stripMax,umapMax,deadMax,restMax))
print("cldMax,clrMax,deadMaxGain,restMaxGain: %d, %d, %.3f, %.3f" % (cldMax,clrMax,deadMaxGain,restMaxGain))
g1 = restMaxGain*ratio_thresh;         g2 = (g1+ratio_thresh);         g3 = 1 - g1;         g4 = 1 - g2
t1 = int(g1*clrMax);      t2 = int(g2*clrMax);      t3 = int(g3*clrMax);      t4 = int(g4*clrMax)
t21 = int(g1*base);       t22 = int(g2*base);       t23 = int(g3*base);       t24 = int(g4*base)
t31 = int((g2+g2*restMaxGain)*clrMax)
print; print("g1,g2,g3,g4:\t%.3f, %.3f, %.3f, %.3f" % (g1,g2,g3,g4));
print("t1,t2,t3,t4:\t%d, %d, %d, %d" % (t1,t2,t3,t4))
print("t21,t22,t23,t24,t31:\t%d, %d, %d, %d, %d" % (t21,t22,t23,t24,t31));print

# _,stripR = cv2.threshold(clR, threshR,255,cv2.THRESH_TOZERO)
_,stripR = cv2.threshold(clR, t31,255,cv2.THRESH_TOZERO)
# _,stripR = cv2.threshold(clR, 43,255,cv2.THRESH_TOZERO)

# pplot(stripD,"stripD")
# pplot(stripR,"stripR")

# Morphological Transformations
newStrip = np.concatenate((stripD,stripR), axis=0); ksize = (5,1)
kernel = cv2.getStructuringElement(cv2.MORPH_RECT,ksize); pStrip = cv2.morphologyEx(newStrip, cv2.MORPH_CLOSE, kernel)
# Find Contours
contours,_ = vboat.find_contours(pStrip, 50.0)
disp = cv2.applyColorMap(pStrip,cv2.COLORMAP_PARULA);       disp = cv2.cvtColor(disp,cv2.COLOR_BGR2RGB)
[cv2.drawContours(disp, [cnt], 0, (255,0,0), 1) for cnt in contours]
# Plots
pplots([stripD,stripR,copyCld,copyClr],"CLAHE Sections",(4,1))
pplots([strip,newStrip,disp],"Test Compare",(3,1))

('strip.shape: ', (85, 640))
Shapes -- deadStrip, restStrip: (28, 640), (57, 640)
tfa: 127
dCl Shape: (3, 640)
[DEBUG] test_filter_first_umap_strip() -----  Shapes: pc1 [(1, 640)], pc2 [(2, 640)]
relRatio, dmRatio,rmRatio: 1.000, 1.000, 0.090
[Deadzone Filtering] base, gain, offset, thresh: 35, 0.00, 0, 35
newStripMax, newRmRatio: 208, 0.111
stripMax,umapMax,deadMax,restMax: 255, 255, 255, 23
cldMax,clrMax,deadMaxGain,restMaxGain: 255, 125, 0.000, 0.816

g1,g2,g3,g4:	0.204, 0.454, 0.796, 0.546
t1,t2,t3,t4:	25, 56, 99, 68
t21,t22,t23,t24,t31:	13, 29, 50, 34, 103



In [55]:
pplot(umap,"umap")
print(umap.shape[0]/2)

27


In [69]:
dm = vboat.dmax
pd = vboat.dmax/256.0
pd = 100/256.0
tmpN = pd*6
print(pd)
print(tmpN)
tmp = math.ceil(tmpN)
print(tmp)

0.390625
2.34375
3.0


In [ ]:
del(display)

In [ ]:
verbose = True;     n = 6
umap = np.copy(vboat.umap_deadzoned)
stripsU = strip_image(umap, nstrips=n)
strip = np.copy(stripsU[1])
tstrip = np.copy(strip)

hs, ws = strip.shape[:2];               dh = hs / n;    umapMax = np.max(umap);     stripMax = np.max(strip)
relRatio = stripMax/float(umapMax)
threshD = int(0.25*stripMax)
print("relRatio, stripMax,threshD: %.3f, %d, %d" % (relRatio,stripMax,threshD))

# if not vboat.is_ground_present: deadThresh = deadThreshOffset
# else: deadThresh = base_thresh + deadThreshOffset

tstrip[tstrip < threshD] = 0
# dead_strip[dead_strip < deadThresh] = 0

# if verbose: print("[Deadzone Filtering] base, gain, offset, thresh: %d, %.2f, %d, %d" % (base_thresh,deadThreshGain,deadThreshOffset,deadThresh))

# Perform CLAHE
clahe = cv2.createCLAHE(clipLimit=2.0,tileGridSize=(hs,ws))
clD = clahe.apply(strip);         copyCld = np.copy(clD)
# _,stripD = cv2.threshold(clD, threshD,255,cv2.THRESH_TOZERO)
cldMax = np.max(clD)
# if deadMax == 0: newDeadMax = 1
# else: newDeadMax = deadMax
deadMaxGain = (cldMax - stripMax)/float(cldMax)
newStripMax = int(stripMax*deadMaxGain)
print("newStripMax: %d" % (newStripMax))
# Remainder Filtering
ratio_thresh = 0.25;   base = 64
# print("stripMax,umapMax,deadMax,restMax: %d, %d, %d, %d" % (stripMax,umapMax,deadMax,restMax))
# print("cldMax,clrMax,deadMaxGain,restMaxGain: %d, %d, %.3f, %.3f" % (cldMax,clrMax,deadMaxGain,restMaxGain))
g1 = deadMaxGain*ratio_thresh;         g2 = (g1+ratio_thresh);         g3 = 1 - g1;         g4 = 1 - g2
t1 = int(g1*cldMax);      t2 = int(g2*cldMax);      t3 = int(g3*cldMax);      t4 = int(g4*cldMax)
t21 = int(g1*base);       t22 = int(g2*base);       t23 = int(g3*base);       t24 = int(g4*base)
t31 = int(g1*t2);
print; print("g1,g2,g3,g4:\t%.3f, %.3f, %.3f, %.3f" % (g1,g2,g3,g4));
print("t1,t2,t3,t4:\t%d, %d, %d, %d" % (t1,t2,t3,t4))
print("t21,t22,t23,t24,t31:\t%d, %d, %d, %d, %d" % (t21,t22,t23,t24,t31));print

# _,stripR = cv2.threshold(clR, threshR,255,cv2.THRESH_TOZERO)
# _,stripR = cv2.threshold(clR, t2-t31,255,cv2.THRESH_TOZERO)
_,newStrip = cv2.threshold(clD, 90,255,cv2.THRESH_TOZERO)

# pplot(stripR,"stripR")

# Morphological Transformations
ksize = (5,1)
# newStrip = np.concatenate((stripD,stripR), axis=0); ksize = (5,1)
kernel = cv2.getStructuringElement(cv2.MORPH_RECT,ksize); pStrip = cv2.morphologyEx(newStrip, cv2.MORPH_CLOSE, kernel)
# Find Contours
contours = vboat.find_contours(pStrip, 50.0)
disp = cv2.applyColorMap(pStrip,cv2.COLORMAP_PARULA);       disp = cv2.cvtColor(disp,cv2.COLOR_BGR2RGB)
[cv2.drawContours(disp, [cnt], 0, (255,0,0), 1) for cnt in contours]
# Plots
pplots([strip,tstrip,newStrip,disp],"Sections U Strip 1",(4,1))
pplots([strip,newStrip,disp],"Test Compare UStrip 1",(3,1))

In [ ]:
verbose = False;     n = 6
vmap = np.copy(vboat.vmap_raw)
slab = np.ones((vmap.shape[0],1),dtype=np.uint8)*255

# tmpV = cv2.cvtColor(vmap, cv2.COLOR_GRAY2BGR)
mask_inv = cv2.bitwise_not(vboat.vmask)
newMask = np.concatenate((mask_inv,slab), axis=1)
# pplot(vmap,"vmap")
# pplot(newMask,"newMask")
print(vmap.shape,newMask.shape)
vmap = cv2.bitwise_and(vmap,vmap,mask=newMask)
stripsV = strip_image(vmap, nstrips=n, horizontal_strips=False);
strips = strip_image(stripsV[0], nstrips=n, horizontal_strips=False);
stripv = np.copy(stripsV[0])
pplots(stripsV,"vstrips",size=(1,6))
# Stripping
hs, ws = stripv.shape[:2];              dh = hs / n
print("hs, ws: %d, %d" % ( hs, ws))
# Initial Stats
vmapMax = np.max(vmap); stripMax = np.max(stripv)
relRatio = stripMax/float(vmapMax)

print("vmapMax, stripMax, relRatio: %d, %d, %.3f" % (vmapMax, stripMax, relRatio))
# Deadzone Filtering
threshD = 7;         base_thresh = 35
# deadThreshGain = (stripMax - deadMax)/255.0;        deadThreshOffset = int(base_thresh*deadThreshGain)
# deadThresh = base_thresh + deadThreshOffset;        dead_strip[dead_strip < deadThresh] = 0
# if verbose: print("[Deadzone Filtering] base, gain, offset, thresh: %d, %.2f, %d, %d" % (base_thresh,deadThreshGain,deadThreshOffset,deadThresh))
# deadMax = np.max(dead_strip)

stripv[stripv < 1] = 0
pplot(stripv,"stripv")
# Perform CLAHE
clahe = cv2.createCLAHE(clipLimit=2.0,tileGridSize=(hs/6,ws/6))
cl = clahe.apply(stripv);         copyCl = np.copy(cl)
clMax = np.max(cl)
_,stripF = cv2.threshold(cl, 64,255,cv2.THRESH_TOZERO)                # Filter CLAHE Deadzone

# if deadMax == 0: newDeadMax = 1
# else: newDeadMax = deadMax
# deadMaxGain = (cldMax - deadMax)/float(cldMax)
# restMaxGain = (clrMax - restMax)/float(clrMax)
# newStripMax = int(stripMax*restMaxGain);        newRmRatio = (restMax)/float(newStripMax)
# print("newStripMax, newRmRatio: %d, %.3f" % (newStripMax,newRmRatio))
# # Remainder Filtering
# ratio_thresh = 0.25;   base = 64
# print("stripMax,umapMax,deadMax,restMax: %d, %d, %d, %d" % (stripMax,umapMax,deadMax,restMax))
# print("cldMax,clrMax,deadMaxGain,restMaxGain: %d, %d, %.3f, %.3f" % (cldMax,clrMax,deadMaxGain,restMaxGain))
# g1 = restMaxGain*ratio_thresh;         g2 = (g1+ratio_thresh);         g3 = 1 - g1;         g4 = 1 - g2
# t1 = int(g1*clrMax);      t2 = int(g2*clrMax);      t3 = int(g3*clrMax);      t4 = int(g4*clrMax)
# t21 = int(g1*base);       t22 = int(g2*base);       t23 = int(g3*base);       t24 = int(g4*base)
# t31 = int(g1*t2);
# print; print("g1,g2,g3,g4:\t%.3f, %.3f, %.3f, %.3f" % (g1,g2,g3,g4));
# print("t1,t2,t3,t4:\t%d, %d, %d, %d" % (t1,t2,t3,t4))
# print("t21,t22,t23,t24,t31:\t%d, %d, %d, %d, %d" % (t21,t22,t23,t24,t31));print

# # _,stripR = cv2.threshold(clR, threshR,255,cv2.THRESH_TOZERO)
# _,stripF = cv2.threshold(clR, t2-t31,255,cv2.THRESH_TOZERO)
# # _,stripR = cv2.threshold(clR, 43,255,cv2.THRESH_TOZERO)



# Morphological Transformations
# newStrip = np.concatenate((stripD,stripR), axis=0); ksize = (5,1)
# kernel = cv2.getStructuringElement(cv2.MORPH_RECT,ksize); pStrip = cv2.morphologyEx(newStrip, cv2.MORPH_CLOSE, kernel)
# # Find Contours
# contours = vboat.find_contours(pStrip, 50.0)
# disp = cv2.applyColorMap(pStrip,cv2.COLORMAP_PARULA);       disp = cv2.cvtColor(disp,cv2.COLOR_BGR2RGB)
# [cv2.drawContours(disp, [cnt], 0, (255,0,0), 1) for cnt in contours]
# Plots
pplots([stripv,cl,stripF],"CLAHE V Sections",(1,3))
# pplots([strip,newStrip,disp],"Test Compare",(3,1))

In [ ]:
# display(Math(r'$$f(a) = \lim_{{x \to a}} \frac{{f(x) - f(a)}}{{x - a}} = {}$$'))
# r1 = (r'$$relRatio = \frac{{stripMax}}{{umapMax}} = {}$$'.format(relRatio))
# r2 = (r'$$dmRatio = \frac{{deadMax}}{{stripMax}} = {}$$'.format(dmRatio))
# r3 = (r'$$restMax = \frac{{restMax}}{{stripMax}} = {}$$'.format(rmRatio))
# tmp = Math(r'{}{}{}'.format(r1,r2,r3))
tmp = Math(r'$$relRatio = \frac{{strip}}{{umap}} =\frac{a}{b}= {}\qquad dmRatio = \frac{{deadzone}}{{strip}} = \frac{}{} = {}\qquad rmRatio = \frac{{rest}}{{strip}} = \frac{}{} = {}$$'.format(a=stripMax,b=umapMax,round(relRatio,3),deadMax,stripMax,round(dmRatio,3),restMax,stripMax,round(rmRatio,3)))
display(tmp)
# display(r1)
# display(r2)
# display(r3)
# r4 = Math(r'$$rmRatio = \frac{{stripMax}}{{umapMax}} = {}$$')

In [ ]:
ustr = np.copy(stripsU[0])
tmpMax = np.max(ustr)

dh = ustr.shape[0]/ 6
drest = (ustr.shape[0]-dh)

dead_strip = ustr[0:dh, :]
orig = np.copy(dead_strip)
rest_strip = ustr[dh:ustr.shape[0], :]

rMax = np.max(dead_strip)

baseDeadThresh = 35
deadThreshGain = (tmpMax - rMax)/255.0
deadThreshOffset = int(baseDeadThresh*deadThreshGain)
deadThresh = baseDeadThresh + deadThreshOffset

# print("base, gain, offset, thresh: %d, %.2f, %d, %d" % (baseDeadThresh,deadThreshGain,deadThreshOffset,deadThresh))

dead_strip[dead_strip < deadThresh] = 0
dClahe = cv2.createCLAHE(clipLimit=2.0,tileGridSize=(dead_strip.shape[0],dead_strip.shape[1]))
dcl = dClahe.apply(dead_strip)
dcl2 = np.copy(dcl)
dcl2[dcl2 < 7] = 0
pplots([orig,dead_strip,dcl,dcl2],"Test",(4,1))

hs,ws = rest_strip.shape[:2]

timg2 = np.copy(rest_strip)
clahe2 = cv2.createCLAHE(clipLimit=2.0,tileGridSize=(timg2.shape[0],timg2.shape[1]))
cl2 = clahe2.apply(timg2)
testMax2 = np.max(cl2)
testThresh2 = int(0.5 * testMax2)

# print("testMax2, testThresh2: %d, %d" % (testMax2,testThresh2))

_,testStrip2 = cv2.threshold(cl2, testThresh2,255,cv2.THRESH_TOZERO)
newStrip2 = np.concatenate((dcl2,testStrip2), axis=0)
pplots([stripsU[0],newStrip2],"CLAHE Compare",(2,1))

In [ ]:
print(params[0].keys())
print(params[0]['absMaxs'][1])

In [ ]:
np.set_printoptions(linewidth=90)
gs = []; ofs = []; vs = []; errs = []; ts=[]
rms = []; ms = []
for p in params:
    tmpSm = p['relMaxs'][0]
    tmpM = p['absMaxs'][1]
    trueThresh = p['threshs'][0]
    
    tmpG = (tmpM - tmpSm)/255.0
#     tmpOf = int(tmpG*tmpSm)
    tmpOf = int(tmpG*b)
    tmpV = b + tmpOf
    err = trueThresh-tmpV
    
    rms.append(tmpSm)
    ms.append(tmpM)
    ts.append(trueThresh)
    gs.append(tmpG)
    ofs.append(tmpOf)
    vs.append(tmpV)
    errs.append(err)

ms = np.array(ms,dtype=int)
rms = np.array(rms,dtype=int)
errs = np.array(errs,dtype=int)
vs = np.array(vs,dtype=int)
ts = np.array(ts,dtype=int)
# print(errs)
# print(rms.tolist())
# print(vs.tolist())
# print(ts.tolist())

plist("Absmax:\t",ms.tolist())
plist("max:\t",rms.tolist())
plist("est:\t",vs.tolist())
plist("true:\t",ts.tolist())

# plist("threshs: ", vs, sep=',')
# plist("errors: ", errs)

In [ ]:
m = 193
sm = 114
b = 35
g = m - sm
g = g/255.0

of = int(g*b)
v = b + of

print(g,of,v)

In [ ]:
threshes = [5, 5, 5, 1,4,5]

tmpdict = {
    "file": imgPathSelector.value,
    "absMaxs" : absMaxs,
    "relMaxs": relMaxs,
    "avgVals": avgVals,
    "threshs": threshes
}
pp.pprint(tmpdict)
keyParams.append(tmpdict)

In [ ]:
# keyParams = []

In [ ]:
svFile = "/home/hunter/data/vboats/test_keyParams/keyParams.pkl"

In [ ]:
svFile = "/home/hunter/data/vboats/test_keyParams/keyParams.pkl"
save_pickle("/home/hunter/data/vboats/test_keyParams/keyParams.pkl",keyParams)

In [ ]:
params = pickle.load(open(svFile, "rb" ))
# print(len(params))
newParams = [d for d in params]
# print(len(newParams))

In [ ]:
tmpParams = pickle.load(open("/home/hunter/data/vboats/test_keyParams/modThreshParams.pkl", "rb" ))

In [ ]:
i = 19
newStrips = []
# pp.pprint(params[i])
threshsU = tmpParams[i]['threshs']
relMaxs = tmpParams[i]['relMaxs']
img_path = tmpParams[i]['file']
plist("threshsU: ",threshsU)
plist("relMaxs: ",relMaxs)
n = len(threshsU); dead_y = 3;
print(i)

# threshsU[0] = 5
# threshsU[1] = 14
# threshsU[2] = 3
# threshsU[3] = 11
# threshsU[4] = 5
# threshsU[5] = 11


img = vboat.read_image(img_path)
pplot(img,"Img")
raw_umap, _, _ = vboat.get_uv_map(img)

cv2.rectangle(raw_umap,(0,0),(raw_umap.shape[1],dead_y),(0,0,0), cv2.FILLED)
cv2.rectangle(raw_umap,(0,raw_umap.shape[0]-dead_y),(raw_umap.shape[1],raw_umap.shape[0]),(0,0,0), cv2.FILLED)

stripsU = strip_image(raw_umap, nstrips=n)
strips = strip_image(stripsU[0], nstrips=n)

for i, strip in enumerate(strips):
    tmpThresh = threshsU[i]
    threshs.append(tmpThresh)
    _,tmpStrip = cv2.threshold(strip, tmpThresh,255,cv2.THRESH_TOZERO)  
    newStrips.append(tmpStrip)
    
newUmap = np.concatenate(newStrips, axis=0)

kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(15,2))
pUmap = cv2.morphologyEx(newUmap, cv2.MORPH_CLOSE, kernel)

pplots(newStrips,"Thresholded Umap 0", flag_resize=True,scale=(5,1))
pplots(strips,"Raw Umap 0", flag_resize=True,scale=(5,1))
pplots([stripsU[0],newUmap],"Umap Compare",(2,1))

contours = vboat.find_contours(newUmap, 25.0)
disp = cv2.applyColorMap(newUmap,cv2.COLORMAP_PARULA)
disp = cv2.cvtColor(disp,cv2.COLOR_BGR2RGB)
[cv2.drawContours(disp, [cnt], 0, (255,0,0), 1) for cnt in contours]
pplot(disp,"Umap")

In [ ]:
params1 = pickle.load(open("/home/hunter/data/vboats/test_keyParams/modThreshParams.pkl", "rb" ))
params2 = pickle.load(open("/home/hunter/data/vboats/test_keyParams/keyParams.pkl", "rb" ))
for i in range(len(params1)):
#     print(len(params1[i]),len(params2[i]))
    print(params1[i]['file']+"\r\n"+params2[i]['file'])
    print

In [ ]:
newParams[0]['threshs'][0] = 60
newParams[0]['threshs'][1] = 16

newParams[2]['threshs'][1] = 14

newParams[3]['threshs'][2] = 12
newParams[3]['threshs'][3] = 10
newParams[3]['threshs'][4] = 13
newParams[3]['threshs'][5] = 12

newParams[4]['threshs'][0] = 33
newParams[4]['threshs'][4] = 7
newParams[4]['threshs'][5] = 11

newParams[5]['threshs'][0] = 35

newParams[6]['threshs'][3] = 18
newParams[6]['threshs'][4] = 12

newParams[12]['threshs'][4] = 3

newParams[14]['threshs'][2] = 3

newParams[15]['threshs'][3] = 7
newParams[15]['threshs'][4] = 5

newParams[16]['threshs'][2] = 3
newParams[16]['threshs'][3] = 11

newParams[17]['threshs'][4] = 5

newParams[20]['threshs'][0] = 5

# pp.pprint(params[i])

In [ ]:
newSvFile = "/home/hunter/data/vboats/test_keyParams/modifiedThreshParams.pkl"
f = open(newSvFile,"wb")
pickle.dump(newParams,f)
f.close()

In [ ]:
testParams = [d for d in tmpParams]
for i, d in enumerate(tmpParams):
    newFile = "params_test_frame_" + str(i) + ".png"
    new_name = "/home/hunter/data/vboats/test_keyParams/" + newFile
    img_path = d['file']
#     tmp = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
#     cv2.imwrite(new_name,tmp)
    testParams[i]['file'] = newFile
    
f = open("/home/hunter/data/vboats/test_keyParams/keyParams.pkl","wb")
pickle.dump(testParams,f)
f.close()

In [ ]:
newParams = []

In [ ]:
# count = 0
# done = False
# while(not done):
#     data = getpass.getpass("")
#     sys.stdout.flush()
#     if data == 'o': count +=1
#     elif data == 'p': count -=1
#     elif data == 'q': done = True

In [ ]:
N = len(baseThreshs)
tests = ['Abs Max', 'Major Strip Max', 'Minor Strip Max']

plist("absMaxs:\t",absMaxs)
print
plist("baseThreshs:\t",baseThreshs,sep=',\t')
plist("absRatios:\t",absRatios,sep=',\t')
plist("relMaxs:\t",relMaxs,sep=',\t')
plist("avgVals:\t",avgVals,sep=',\t')
print

absMax = absMaxs[0]; strMax = absMaxs[1]

relRatios = []; relRatios2 = []; avgRatios = []; avgRatios2 = [];
ratios = []; maxs = []; gains = [];
dratio = dmax/255.0
for i in range(N):
    maxs.append([absMax, strMax,relMaxs[i]])
    relRatio = (absMax-relMaxs[i])/float(absMax)
    relRatio2 = (strMax-relMaxs[i])/float(strMax)
    avgRatio = (absMax-avgVals[i])/float(absMax)
    avgRatio2 = (strMax-avgVals[i])/float(strMax)
    
    relRatios.append(relRatio)
    relRatios2.append(relRatio2)
    avgRatios.append(avgRatio)
    avgRatios2.append(avgRatio2)
    
    ratios.append([relRatio,relRatio2,avgRatio,avgRatio2,1-relRatio,1-relRatio2,1-avgRatio,1-avgRatio2])
maxs = np.array(maxs)
_,nMaxs = maxs.shape[:2]
ratios = np.array(ratios).T
ntests,nstrips = ratios.shape[:2]

ratio_lbls = ["Ratios "+str(t) for t in range(ntests)]
gain_lbls = ["Gains "+str(t) for t in range(ntests)]
thresh_lbls = ["Thresholds "+str(t) for t in range(ntests)]

test_lbls = ["Test: "+str(test) for test in tests]
col_lbls = ["Strip "+str(strip) for strip in range(nstrips)]
val_lbls = [str(test) for test in tests]

threshs = np.zeros((ntests,nstrips,nMaxs), dtype=np.uint8)
for ch in range(nMaxs):
    for test in range(ntests):
        tmpGains = []
        for strip in range(nstrips):
            tmpGain = baseThreshs[strip]*ratios[test,strip]
            tmpGains.append(tmpGain)
            tmpThresh = tmpGain * maxs[strip,ch]
            threshs[test,strip,ch] = tmpThresh
        if ch == 0: gains.append(tmpGains) 
gains = np.array(gains)

pmat(maxs,val_lbls,col_lbls)
pmat(ratios,col_lbls,ratio_lbls)
pmat(gains,col_lbls,gain_lbls)#,"\r\nGains")

for i, lbl in enumerate(test_lbls):
    tThreshs = threshs[:,:,i]
    pmat(tThreshs,col_lbls,thresh_lbls,lbl)

In [ ]:
pklParentDir = "/home/hyoung/data/vboats/test_keyParams"
pklFile = os.path.join(pklParentDir,"keyParams.pkl")
params = load_pickle(pklFile)
print(params[0].keys())
# pp.pprint([p['relMaxs'][0] for p in params])

In [ ]:
# pp.pprint(params)
tmp = params[0]
tmp['relMaxs']
# df = DataFrame(tmp,columns=tmp.keys())
# df

In [ ]:
temp = vboat.umap_deadzoned
tempv = vboat.vmap_raw

tmpStrs = strip_image(temp, nstrips=6)
tmp0Strs = strip_image(tmpStrs[0], nstrips=6)

tmpStrsv = strip_image(tempv, nstrips=6,horizontal_strips=False)
tmpv0Strs = strip_image(tmpStrsv[0], nstrips=6,horizontal_strips=False)

pplots(tmp0Strs,"Umap 0", flag_resize=True,scale=(5,1))
pplots(tmpv0Strs,"vmap strip",(1,6),flag_resize=True,scale=(1,10))
pplots(tmpStrs,"umap strips")

tmps = [vboat.umap_deadzoned,vboat.umap_processed]
pplots(tmps,"umaps",(len(tmps),1))

In [ ]:
print(vboat.umap_deadzoned.shape,vboat.umap_processed.shape)

In [ ]:
cnts = np.array([cv2.arcLength(cnt,True) for cnt in vboat.contours])
print(np.unique(cnts))

In [ ]:
tmpmax = dmax
# rat = tmpmax/255.0
tmpt = threshsU[0]

rat = 187/255.0

gain = tmpt*rat*10
print(tmpmax,rat, gain,gain*61)

In [ ]:
hm = cv2.imread("/home/hyoung/data/HM.png", cv2.IMREAD_GRAYSCALE)
noise_img = cv2.imread("/home/hyoung/data/R.png", cv2.IMREAD_GRAYSCALE)
pplot(hm,"raw HM")
pplot(noise_img,"noise")
scaled = noise_img * 0.1
pplot(scaled,"scaled noise")
composite_heightmap = hm + noise_img
composite_heightmap_scaled = hm + scaled

print(noise_img[0,0],scaled[0,0])

pplot(composite_heightmap,"comp")
cv2.imwrite("/home/hyoung/data/composite.png", composite_heightmap)
cv2.imwrite("/home/hyoung/data/composite_scaled.png", composite_heightmap_scaled)
cv2.imwrite("/home/hyoung/data/scaled_nosie.png", scaled)

In [ ]:
tImg = np.copy(vboat.img)
umap = vboat.umap_raw
vmap = vboat.vmap_raw
deadu = vboat.umap_deadzoned
pv = vboat.vmap_processed
pu = vboat.umap_processed

# pplot(tImg,'Input')
pplot(umap,'Raw Umap')
# pplot(vmap,'Raw Vmap')
pplot(deadu,'Deadzoned Umap')
pplot(pv,'Processed vmap')



# mn = np.mean(tImg)
# st = np.std(tImg[tImg > 10])
# norm = (tImg - mn)/st

# norm = np.expand_dims(norm,axis=-1)

# norm = cv2.normalize(norm,None,0.0,255.0,cv2.NORM_MINMAX)
# norm = np.asarray(norm, dtype=np.uint8)
# nUmap, nVmap,_ = vboat.get_uv_map(norm)

# sub = nUmap - umap 
# pplot(sub,'Sub')

# copy = vmap.copy()
# # copy = cv2.cvtColor(copy,cv2.COLOR_GRAY2BGR)
# copy = np.float32(copy)
# criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
# K = 150
# ret,label,center=cv2.kmeans(copy,K,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)

# # Now convert back into uint8, and make original image
# center = np.uint8(center)
# res = center[label.flatten()]
# res2 = res.reshape((vmap.shape))
# pplot(res2,'kmeans')

# nUmap = nUmap**2
# nUmap[nUmap < np.max(nUmap*0.3)] = 0

# pplot(nUmap,'Norm Umap')

# pplot(vboat.umap_processed,'Processed Umap')

# _, tmpU = cv2.threshold(vmap, 10, 255,cv2.THRESH_TOZERO)
# tmpU = cv2.equalizeHist(tmpU)

# pplot(tmpU,1)

In [ ]:
dispU1,dispU2,compU = vboat.umap_displays()
dispV1,dispV2,compV = vboat.vmap_displays()

# plt.figure(6)
# plt.imshow(vboat.img)
# plt.show()

dispU2 = cv2.cvtColor(dispU2,cv2.COLOR_BGR2RGB)
plt.figure(0)
plt.imshow(dispU2)
plt.show()

# plt.figure(1)
# plt.imshow(dispU2)
# plt.show()

plt.figure(2)
plt.imshow(vboat.vmap_processed**2)
plt.show()

dispV2 = cv2.cvtColor(dispV2,cv2.COLOR_BGR2RGB)
plt.figure(3)
plt.imshow(dispV2)
plt.show()

# plt.figure(4)
# plt.imshow(dispV2)
# plt.show()

# plt.figure(5)
# plt.imshow(compV)
# plt.show()

In [ ]:
tImg = np.copy(vboat.img)

raw_umap, raw_vmap, dt = vboat.get_uv_map(tImg)

# =========================================================================
dead_y = vboat.dead_y; dead_x = vboat.dead_x

deadzoneU = raw_umap[1:dead_y+1, :]
_, deadzoneU = cv2.threshold(deadzoneU, 80, 255,cv2.THRESH_BINARY)

cv2.rectangle(raw_umap,(0,0),(raw_umap.shape[1],dead_y),(0,0,0), cv2.FILLED)
cv2.rectangle(raw_umap,(0,raw_umap.shape[0]-dead_y),(raw_umap.shape[1],raw_umap.shape[0]),(0,0,0), cv2.FILLED)
cv2.rectangle(raw_vmap,(0,0),(dead_x, raw_vmap.shape[0]),(0,0,0), cv2.FILLED)
cv2.rectangle(raw_vmap,(raw_vmap.shape[1]-dead_x,0),(raw_vmap.shape[1],raw_vmap.shape[0]),(0,0,0), cv2.FILLED)
# =========================================================================

threshs = [11, 10, 30, 40,40,40]
stripsPu = []
stripsU = strip_image(raw_umap, nstrips=len(threshs))

strU1 = stripsU[0]
strU2 = stripsU[1]

for i, strip in enumerate(stripsU):
#     _, tmpStrip = cv2.threshold(strip, threshs[i], 255,cv2.THRESH_BINARY)
    _, tmpStrip = cv2.threshold(strip, threshs[i], 255,cv2.THRESH_TOZERO)
    stripsPu.append(tmpStrip)

kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(15,2))
stripsPu[0] = cv2.morphologyEx(stripsPu[0], cv2.MORPH_CLOSE, kernel)

tK = np.ones((3,2),np.uint8)
copy = stripsPu[1].copy()

stripsPu[1] = cv2.morphologyEx(stripsPu[1], cv2.MORPH_CLOSE, kernel)

# stripsPu[1] = cv2.morphologyEx(stripsPu[1], cv2.MORPH_OPEN, (2,2))
# stripsPu[1] = cv2.blur(stripsPu[1],(2,2))
tmpp = cv2.morphologyEx(stripsPu[1], cv2.MORPH_GRADIENT, kernel)
# stripsPu[1] = cv2.equalizeHist(stripsPu[1])
# stripsPu[2] = cv2.morphologyEx(stripsPu[2], cv2.MORPH_OPEN, kernel)
pplot(tmpp, 'tmpp')
stripsPu[1] = cv2.morphologyEx(stripsPu[1], cv2.MORPH_BLACKHAT, kernel)

# ============================ CRAP ============================
kernel = np.ones((4,4),np.uint8)

_, tmpStrip = cv2.threshold(stripsPu[1], 1, 255,cv2.THRESH_BINARY)
# _, tmpStrip2 = cv2.threshold(stripsPu[1], 1, 255,cv2.THRESH_BINARY_INV)
# dist_transform = cv2.distanceTransform(stripsPu[1],cv2.DIST_L2,5)
dist_transform = cv2.distanceTransform(tmpStrip,cv2.DIST_L2,5)
# dist_transform2 = cv2.distanceTransform(tmpStrip2,cv2.DIST_L2,5)
# pplot(dist_transform, 'dist_transform')
# pplot(dist_transform2, 'dist_transform2')

# erode = cv2.dilate(dist_transform2,kernel,iterations=5)
erode = cv2.erode(dist_transform,kernel,iterations=1)
# stripsPu[1] = np.uint8(erode)

# dist_transform = cv2.distanceTransform(stripsPu[1],cv2.DIST_L2,5)
# erode = cv2.erode(dist_transform,kernel,iterations=2)
# _, sure_fg = cv2.threshold(erode,0.1*erode.max(),255,0)

# sure_bg = cv2.dilate(sure_fg,kernel,iterations=1)
# unknown = cv2.subtract(sure_bg,sure_fg)

# # pplot(dist_transform, 'dist_transform')
# # pplot(sure_fg, 'sure_fg')
# # pplot(erode, 'erode')

# sure_fg = np.uint8(sure_fg)
# _, markers = cv2.connectedComponents(sure_fg)


# markers = markers+1
# # Now, mark the region of unknown with zero
# markers[unknown==255] = 0
# pplot(markers, 'markers')

# tmp = cv2.cvtColor(stripsPu[1],cv2.COLOR_GRAY2BGR)
# markers = cv2.watershed(tmp,markers)
# tmp[markers == -1] = [255,0,0]

# pplot(tmp, 'tmp')
# ============================ CRAP ============================


hUs,w = stripsPu[0].shape[:2]
blankStrip = np.zeros((hUs-dead_y,w),dtype=np.uint8)
deadzone_mask = np.concatenate((deadzoneU,blankStrip), axis=0)
# try: deadzone_mask = cv2.cvtColor(deadzone_mask, cv2.COLOR_GRAY2BGR)
# except: print("[WARNING] ------------  Unnecessary Deadzone Image Color Converting")

kernelD = cv2.getStructuringElement(cv2.MORPH_RECT,(50,5))
deadzone_mask = cv2.morphologyEx(deadzone_mask, cv2.MORPH_CLOSE, kernelD)

# print(stripsPu[0].shape, deadzone_mask.shape)
stripsPu[0] = cv2.addWeighted(stripsPu[0], 1.0, deadzone_mask, 1.0, 0)
pplot(stripsPu[0], 'stripsPu[0] After')

tmpMax = np.max(stripsPu[0])
# print(tmpMax)
dead_strip = stripsPu[0][0:14, :]
rest_strip = stripsPu[0][14:stripsPu[0].shape[0], :]

dead_strip[dead_strip < tmpMax*0.4] = 0
rest_strip[rest_strip < tmpMax*0.1] = 0
stripsPu[0] = np.concatenate((dead_strip,rest_strip), axis=0)


# pplot(stripsPu[1], 'stripsPu[1] After')



pplot(deadzone_mask, 'Deadzone Mask')
   
pUmap = np.concatenate(stripsPu, axis=0)

cnt_threshs = [55.0,50.0,80.0,40.0,40.0,40.0]
contours = []
for i, strip in enumerate(stripsPu):
    contours += vboat.find_contours(strip, cnt_threshs[i], offset=(0,hUs*i))
# contours = fCnts1 + fCnts2 + fCnts3 + fCnts4 + fCnts5 + fCnts6
print(len(contours))

disp = pUmap.copy()
for cnt in contours:
    cv2.drawContours(disp, [cnt], 0, (255,0,255), 1)
    
pplot(disp, 'Filtered Umap')

xLims, dLims, _ = vboat.extract_contour_bounds(contours)
# ==============================================================================
#                           V MAP
# ==============================================================================
threshsV = [5, 20, 40,40,40]
stripsV = strip_image(raw_vmap, nstrips=len(threshsV), horizontal_strips=False)

pplot(stripsV[0], 'stripsPv[0] Before')
# pplot(stripsV[1], 'stripsPv[1] Before')

stripsPv = []
for i, strip in enumerate(stripsV):
#     _, tmpStrip = cv2.threshold(strip, threshsV[i], 255,cv2.THRESH_BINARY)
    _, tmpStrip = cv2.threshold(strip, threshsV[i], 255,cv2.THRESH_TOZERO)
    stripsPv.append(tmpStrip)
    
tmpMax = np.max(stripsPv[0])
# print(tmpMax)
stripsPv[0][stripsPv[0] < np.max(stripsPv[0]*0.035)] = 0

top_strip = stripsPv[0][0:100, :]
bot_strip = stripsPv[0][100:stripsPv[0].shape[0], :]
    
top_strip[top_strip < tmpMax*0.1] = 0
bot_strip[bot_strip < tmpMax*0.05] = 0
stripsPv[0] = np.concatenate((top_strip,bot_strip), axis=0)

# tmpMax = np.max(stripsPv[1])
tH = stripsPv[1].shape[0]
top_strip = stripsPv[1][0:tH/2, :]
bot_strip = stripsPv[1][tH/2:tH, :]

top_strip[top_strip < tmpMax] = 0
# bot_strip[bot_strip < tmpMax*0.05] = 0
stripsPv[1] = np.concatenate((top_strip,bot_strip), axis=0)    
    
    
# pplot(stripsPv[0]**2, 'stripsPv[0] After')
# pplot(stripsPv[1]**2, 'stripsPv[1] After')
    
newV = np.concatenate(stripsPv, axis=1)
tmpV = cv2.cvtColor(newV, cv2.COLOR_GRAY2BGR)
mask, maskInv,mPxls, ground_wins,_ = vboat.get_vmap_mask(tmpV, maxStep = 15)
vmap = cv2.bitwise_and(newV,newV,mask=maskInv)
pplot(vmap*10, 'Vmap After')
# =========================================================================

obs, obsU, ybounds, dbounds, windows, nObs = vboat.find_obstacles(vmap, dLims, xLims)

dispObs = cv2.cvtColor(tImg, cv2.COLOR_GRAY2BGR)

for ob in obs:
    cv2.rectangle(dispObs,ob[0],ob[1],(150,0,0),1)
    
print(len(obs))
pplot(dispObs, 'Obstacles')
# pplot(strU1, 'stripsPu[0] Before')
# pplot(strU2, 'stripsPu[1] Before')

In [ ]:
pklParentDir = "/home/hyoung/data/vboats/test_keyParams"
sprImg,fact = spread_image(strU1)
sub1 = strU1 - sprImg[:42,:]
pplot(sub1,'Spread U1')

desprdImg = despread_image(sprImg,fact)
print(desprdImg.shape)
sub2 = strU1 - desprdImg[:42,:]
pplot(sub2,'Spread U2')
pplot(desprdImg,'DeSpread U2')


In [ ]:
ellipses = []
disp = np.copy(vboat.umap_processed)
disp = cv2.cvtColor(disp,cv2.COLOR_GRAY2BGR)
for cnt in vboat.filtered_contours:
    ellipse = cv2.fitEllipse(cnt)
    ellipses.append(ellipse)
    cv2.ellipse(disp,ellipse,(0,255,255),1)
    
plt.figure(1)
plt.imshow(disp)
plt.show()

In [ ]:
img = np.copy(vboat.img)
print(vboat.obstacles[0])
win = vboat.obstacles[0]
print(win[0][0], win[1][0])

rectsTypeA = [] # x0, y0, x1, y1
rectsTypeB = [] # x, y, w, h
for win in vboat.obstacles:
    
    wid = win[1][0] - win[0][0]
    h = win[1][1] - win[0][1]
    cx = win[0][0] + wid/2
    cy = win[0][1] + h/2
    
    tmpA = [win[0][0], win[0][1], win[1][0], win[1][1]]
    tmpB = [cx, cy, wid, h]
    rectsTypeA.append(tmpA)
    rectsTypeB.append(tmpB)


# cv2.circle(img,(pxl[0],pxl[1]),2,(255,0,255),-1)

print(wid, h)
print(cx, cy)
print(rectsTypeB)
print(vboat.nObs)

In [ ]:
n = vboat.nObs
idcs = []
pick = []
boxes = np.array(rectsTypeA)
x1 = boxes[:,0]
y1 = boxes[:,1]
x2 = boxes[:,2]
y2 = boxes[:,3]
area = (x2 - x1 + 1) * (y2 - y1 + 1)
idxs = np.argsort(y2)
overlapThresh = 0.05
supressed = []
top = []
non = []
print(idxs)
while len(idxs) > 0:
    # grab the last index in the indexes list, add the index
    # value to the list of picked indexes, then initialize
    # the suppression list (i.e. indexes that will be deleted)
    # using the last index
    last = len(idxs) - 1
    i = idxs[last]
    pick.append(i)
    suppress = [last]
    # loop over all indexes in the indexes list
    print(i)
    overlaps = False
    for pos in xrange(0, last):
        # grab the current index
        j = idxs[pos]

        # find the largest (x, y) coordinates for the start of
        # the bounding box and the smallest (x, y) coordinates
        # for the end of the bounding box
        xx1 = max(x1[i], x1[j])
        yy1 = max(y1[i], y1[j])
        xx2 = min(x2[i], x2[j])
        yy2 = min(y2[i], y2[j])

        # compute the width and height of the bounding box
        w = max(0, xx2 - xx1 + 1)
        h = max(0, yy2 - yy1 + 1)

        # compute the ratio of overlap between the computed
        # bounding box and the bounding box in the area list
        
        overlap = float(w * h) / area[j]

        print("Window %d Overlaps %.4f with Rectangle %d" % (j,overlap, i) )
        # if there is sufficient overlap, suppress the
        # current bounding box
        if overlap > overlapThresh:
#             suppress.append(pos)
            supressed.append(pos)
            top.append([pos,i])
            overlaps = True

    # delete all indexes from the index list that are in the
    # suppression list
    print("suppressing idxs", suppress)
#     supressed.append(suppress)
    idxs = np.delete(idxs, suppress)
    if(not overlaps):
        non.append(last)

rectangles = [] 
for a,b in top:
    rect = union(rectsTypeA[a],rectsTypeA[b])
    rectangles.append(rect)
    

print(top)
print(non)
print(supressed)
mask = np.isin(non, supressed, invert=True)
remain = np.where(mask)
print(remain)
rectangles.append(rectsTypeA[remain[0][0]])

print(rectangles)
for i in range(n):
    has_overlap = False
    for j in range(n):
        if(i is not j):
            iou = test_intersection(rectsTypeA[i],rectsTypeA[j])
            if(iou >= 0.05):
                has_overlap = True
                idcs.append([i, j])
                print("IoU for Rect %d and Rect %d = %.3f" % (i, j, iou))
    if(not has_overlap):
        print("Rect %d has no overlapping segments" % i)

In [ ]:
disp = np.copy(vboat.img)
disp = cv2.cvtColor(disp,cv2.COLOR_GRAY2BGR)
for x1,y1,x2,y2 in boxes[pick]:
    cv2.rectangle(disp,(x1,y1),(x2,y2),(0,255,255), 1)
    
plt.figure(10)
plt.imshow(disp)
plt.show()

In [ ]:
disp = np.zeros_like(vboat.img)
disp = cv2.cvtColor(disp,cv2.COLOR_GRAY2BGR)
masks = []
for win in vboat.obstacles:
    mask = np.copy(disp)
    cv2.rectangle(mask,win[0],win[1],(255,255,255), cv2.FILLED)
    mask = cv2.cvtColor(mask,cv2.COLOR_BGR2GRAY)
    masks.append(mask)
    

tmp = cv2.bitwise_and(vboat.img,vboat.img, mask=masks[0])

tmpMean = np.mean(tmp)
print(tmpMean/0.001)
plt.figure(2)
plt.imshow(tmp)
plt.show()

np.amin()

In [ ]:
newStrips = []
raw_umap = vboat.umap_raw; dead_y = 3
umap = np.copy(raw_umap)

cv2.rectangle(umap,(0,0),(umap.shape[1],dead_y),(0,0,0), cv2.FILLED)
cv2.rectangle(umap,(0,umap.shape[0]-dead_y),(umap.shape[1],umap.shape[0]),(0,0,0), cv2.FILLED)

threshsU = [0.15, 0.15, 0.075, 0.065,0.05,0.025]
n = len(threshsU)

stripsU = strip_image(umap, nstrips=n)
strips = strip_image(stripsU[0], nstrips=n)
strips2 = strip_image(stripsU[1], nstrips=n)

# print(np.max(umap))
absmax = np.max(umap)
dmax = absmax
stripmax = np.max(stripsU[0])
# print(dmax,stripmax)
baseThreshs = []
absMaxs=[absmax,stripmax]
absRatios = []
relMaxs = []
avgVals = []
threshs = []
for i, strip in enumerate(strips):
    dratio = dmax/255.0
    tmpMax = np.max(strip)
    relRatio = (dmax-tmpMax)/float(dmax)
    avgd = strip[np.nonzero(strip)].mean()
    if np.isnan(avgd):
        avgd = np.nan_to_num(avgd)
    avgRatio = math.fabs(avgd)/float(dmax)
    
    baseThreshs.append(threshsU[i])
    absRatios.append(dratio)
    relMaxs.append(tmpMax)
    avgVals.append(int(avgd))
    
    if(dratio <= 0.35):
        tmpGain = 10.0
        threshGain = 0.6
        idxPerc = (1+i)/float(n)
        if idxPerc > 0.5: threshGain = threshGain + 0.25
#         if i == 0: threshAdder = 0.35
        if i <= 1: threshAdder = math.fabs(relRatio-dratio)
        else: threshAdder = 0
    else:
#         tmpGain = 1.0
        tmpGain = dratio
#         threshGain = 1 - relRatio
        threshGain = 1-avgRatio
        if i == 0: threshAdder = avgRatio #0.15
        else: threshAdder = 0
    
    
    if relRatio == 0: ratioGain = 0
    else: ratioGain = avgRatio#(relRatio)*
        
    if i == 0: gain = threshsU[i]*relRatio+ ratioGain*0# + threshAdder
    else:
        gain = threshsU[i]*threshGain*dratio*tmpGain + threshAdder
        gain = gain * 1.5
    
#     gain = (avgRatio) * gain * 10
#     gain = 1-gain
#     gain = math.fabs(gain)
#     gain = threshsU[i]*threshGain*(math.fabs(relRatio-dratio))*tmpGain + threshAdder

#     testGain = threshsU[i]*threshGain*relRatio*tmpGain + threshAdder
    testGain = threshsU[i]*threshGain*dratio*tmpGain + threshAdder
    testGain = testGain*1.5
    if relRatio == 0: tratioGain = 0
    else: tratioGain = (1-relRatio)
    testGain2 = threshsU[i]+(tratioGain)
    testThresh = int(testGain * dmax)
    testThresh2 = int(testGain2 * dmax)
    tmpThresh = int(gain * dmax)
    threshs.append(tmpThresh)
    
    if relRatio == 0: t2ratioGain = dratio
    else: t2ratioGain = relRatio

#     print(avgd, avgRatio, ratioGain/dratio)
#     print("max, relative max, totalPerc, relativePerc: %d, %d, %.3f, %.3f" % (dmax,tmpMax,dratio,relRatio))
#     print("total gain, relative gain: %.3f, %.3f" % (testGain,testGain2))
#     print("thresh w/ total max, relative max: %d, %d" % (testThresh,testThresh2))
#     print("dratio, relRatio, pre_gain, gain, adder, thresh: %.3f, %.3f, %.3f, %.3f, %.3f, %d" %
#           (dratio,relRatio, threshGain,gain,threshAdder,tmpThresh))
    
#     _,tmpStrip = cv2.threshold(strip, tmpThresh,255,cv2.THRESH_BINARY)
    _,tmpStrip = cv2.threshold(strip, tmpThresh,255,cv2.THRESH_TOZERO)
    
    newStrips.append(tmpStrip)
    
newUmap = np.concatenate(newStrips, axis=0)

ksize = (15,2)
kernel = cv2.getStructuringElement(cv2.MORPH_RECT,ksize)
pUmap = cv2.morphologyEx(newUmap, cv2.MORPH_CLOSE, kernel)

pplots(newStrips,"Thresholded Umap 0", flag_resize=True,scale=(5,1))
pplots(strips,"Raw Umap 0", flag_resize=True,scale=(5,1))
# pplots(strips2,"Raw Umap 1", flag_resize=True,scale=(5,1))

contours = vboat.find_contours(pUmap, 50.0)
disp = cv2.applyColorMap(pUmap,cv2.COLORMAP_PARULA)
disp = cv2.cvtColor(disp,cv2.COLOR_BGR2RGB)
[cv2.drawContours(disp, [cnt], 0, (255,0,0), 1) for cnt in contours]
pplot(disp,"Umap")
pplot(overlay,'Overlay')

plist("absMaxs: ", absMaxs)
plist("relMaxs: ", relMaxs)
plist("threshs: ", threshs)
plist("avgVals: ", avgVals)